

# Function calling
We strongly recommend checking the official [Function-calling](https://platform.openai.com/docs/guides/function-calling) page for the most up-to-date information.

#### Overview
Objective: more reliably get structured data back from the model
- Create assistants that answer questions by calling external APIs (e.g. like ChatGPT Plugins)
    - send_email(to: string, body: string)
    - get_current_weather(location: string, unit: 'celsius' | 'fahrenheit')
- Convert natural language into API calls
    - get_customers(min_revenue: int, created_before: string, limit: int) from internal API
- Extract structured data from text
    - extract_data(name: string, birthday: string)
    - sql_query(query: string)

#### Basics
- **USER**: describe the function
- **USER**: call the model with user query + function parameter
- **MODEL**: choose to call one or more functions and generate a JSON object with function arguments
- **USER**: call your function with the provided arguments 
- **USER**: call the model again with the function response in query

#### Reference
Full detailed information about function-calling can be found in the OpenAI cookbook for ["How to call functions with chat models"](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).   

#### Flight Status Query Project 
We demonstrate how function-call can be adopted in this example.


In [1]:
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

- Step 0: Wrote helper function to get flight status

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# Python code for flight status is adapted from 
# https://www.tutorialspoint.com/get-flight-status-using-python
#
# Note: Tracking is available for flights scheduled 3 days before or after today.
#
def get_flight_status(airline_code, flight_number, day, month, year):
    def get_data(url):
        response = requests.get(url)
        return response.text

    url = f"https://www.flightstats.com/v2/flight-tracker/{airline_code}/{flight_number}?year={year}&month={month}&date={day}"
    html_data = get_data(url)

    soup = BeautifulSoup(html_data, 'html.parser')

    statuses = [
        item.get_text() for item in soup.find_all("div", class_="text-helper__TextHelper-sc-8bko4a-0 feVjck")
    ]
    time_statuses = [
        item.get_text() for item in soup.find_all("div", class_="text-helper__TextHelper-sc-8bko4a-0 kbHzdx")
    ]

    return str(statuses[0] + "; Departing at " + time_statuses[0] + "; Arriving at " + time_statuses[2])

- Step 1: We asked GPT model about flight status, supplying function name and details

In [3]:
response = client.chat.completions.create(
  model = "gpt-3.5-turbo-0613",
  messages = [
    {
      "role": "user",
      "content": "What is the flight status of UA 792 for Nov 12, 2023?"
    }
  ],
  functions = [
    {
      "name": "get_flight_status",
      "description": "Get the current flight status",
      "parameters": {
          "type": "object",
          "properties": {
              "airline_code": {
                  "type": "string",
                  "description": "Shorthand notation for airline, e.g. UA, AA"
              },
              "flight_number": {
                  "type": "integer",
                  "description": "Flight number"
              },
              "day": {
                  "type": "integer",
                  "description": "Day of the flight"
              },
              "month": {
                  "type": "integer",
                  "description": "Month of the flight"
              },
              "year": {
                  "type": "integer",
                  "description": "Year of the flight"
              },
            },
          "required": ["airline_code","flight_number","day","month","year"]
        }
    }
  ]
)
print(response)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-3.5-turbo-0613` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

- Step 2: GPT responded with function call arguments. We parsed the response and ran our helper function 

In [16]:
import json
function_name = response.choices[0].message.function_call.name
arguments = response.choices[0].message.function_call.arguments
# Read arguments to dictionary
arguments = json.loads(arguments)
# Call helper function
function_response_msg = locals()[function_name](arguments["airline_code"], arguments["flight_number"], arguments["day"], arguments["month"], arguments["year"])
print(function_response_msg)

On time; Departing at 06:00 CST; Arriving at 09:06 EST


- Step 3: We added helper function output to GPT query and called GPT model for the final response

In [17]:
response_2 = client.chat.completions.create(
  model = "gpt-3.5-turbo-0613",
  messages = [
    {
      "role": "user",
      "content": "What is the flight status of UA 792 for Nov 12, 2023?"
    },
    {
      "role": "function",
      "name": "get_flight_status",
      "content": function_response_msg
    }
  ],
  functions = [
    {
      "name": "get_flight_status",
      "description": "Get the flight status of flight for the specified date",
      "parameters": {
          "type": "object",
          "properties": {
              "airline_code": {
                  "type": "string",
                  "description": "Shorthand notation for airline, e.g. UA, AA"
              },
              "flight_number": {
                  "type": "integer",
                  "description": "Flight number"
              },
              "day": {
                  "type": "integer",
                  "description": "Day of the flight"
              },
              "month": {
                  "type": "integer",
                  "description": "Month of the flight"
              },
              "year": {
                  "type": "integer",
                  "description": "Year of the flight"
              },
            },
          "required": ["airline_code","flight_number","day","month","year"]
        }
    }
  ]
)
print(response_2)

ChatCompletion(id='chatcmpl-8KWdNwNEKsRQDoXLJ6cnrYEBnnRau', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The flight status of UA 792 for November 12, 2023, is on time. The flight is scheduled to depart at 06:00 CST and arrive at 09:06 EST.', role='assistant', function_call=None, tool_calls=None))], created=1699902697, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=42, prompt_tokens=150, total_tokens=192))


- Now GPT can tell you the flight status!

In [18]:
print(response_2.choices[0].message.content)

The flight status of UA 792 for November 12, 2023, is on time. The flight is scheduled to depart at 06:00 CST and arrive at 09:06 EST.
